In [290]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

auth_data = {
    'grant_type'    : 'client_credentials',
    'client_id'     : '18d9f9c339ef48b4889c49373ecdfd96',
    'client_secret' : 'b5158ce607b3ef1a548ead081fd8ac44d6ee986cb2a6f49624611d8f962bb51b',
    'scope'         : 'read_product_data read_financial_data read_content'
}

# create Session instance
session = requests.Session()

# make a POST to retrieve access_token
auth_request = session.post('https://idfs.gs.com/as/token.oauth2', data = auth_data)
access_token_dict = json.loads(auth_request.text)
access_token = access_token_dict['access_token']

# update session headers
session.headers.update({'Authorization':'Bearer '+ access_token})

print(access_token)

8cQPnK9XSRqJOasTyiU2LypPBubP


In [291]:
import pandas as pd
request_url = 'https://api.marquee.gs.com/v1/data/USCANFPP_MINI/coverage?limit=100'
request = session.get(url=request_url)
data = json.loads(request.text)
df = pd.DataFrame(data['results'])
gsids = df.gsid
# print(gsids)

In [301]:
payload = {
    "startDate": "2012-07-02",
    "endDate": "2017-06-30",
    "where": {
        "gsid": list(gsids.values)
    }
}

request_url = 'https://api.marquee.gs.com/v1/data/USCANFPP_MINI/query'
request = session.post(url=request_url, json = payload)
results = json.loads(request.text)
data = results['data']
df = pd.DataFrame(data)

In [294]:
import pandas as pd
request_url = 'https://api.marquee.gs.com/v1/data/USCANFPP_MINI/coverage?limit=100'
request = session.get(url=request_url)
data = json.loads(request.text)
df = pd.DataFrame(data['results'])
gsids = df.gsid
#print(gsids)
payload = {
    "where": {
        "gsid": list(gsids.values)
    },
    "fields": [ "gsid", "ticker", "name" ],
    "limit": 1000
}

request_url = 'https://api.marquee.gs.com/v1/assets/data/query'
request = session.post(url=request_url, json = payload)
results = json.loads(request.text)
data = results['results']
# print(data)
ticker = pd.DataFrame(data)['ticker'].values
gsid = pd.DataFrame(data)['gsid'].values

stockprice = pd.DataFrame()
for i in np.unique(ticker):
    temp_gsid = np.unique(gsid[i == ticker])[0]
    #print(temp_gsid)
    request_url = 'https://api.iextrading.com/1.0/stock/' + i + '/chart/5y'
    request = session.get(url=request_url)
    results = json.loads(request.text)
    stockData = pd.DataFrame(results)
    stockData['ticker'] = i
    stockData['gsid'] = temp_gsid
    stockprice = stockprice.append(stockData)
    #print(stockData)

In [295]:
stockprice

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,ticker,gsid
0,-1.488600,0.000000,-2.421,60.0071,2013-03-04,61.1714,"Mar 4, 13",59.8571,61.1142,20772338,145406366,60.4793,AAPL,14593
1,1.584900,0.026410,2.641,61.5919,2013-03-05,62.1699,"Mar 5, 13",60.1071,60.2114,22756860,159298020,61.3354,AAPL,14593
2,-0.783142,0.013360,-1.272,60.8088,2013-03-06,62.1785,"Mar 6, 13",60.6328,62.0728,16414740,114903180,61.1082,AAPL,14593
3,0.702856,0.025074,1.156,61.5117,2013-03-07,61.7157,"Mar 7, 13",60.1514,60.6428,16713263,116992841,60.9832,AAPL,14593
4,0.162571,0.027782,0.264,61.6742,2013-03-08,62.2042,"Mar 8, 13",61.2299,61.3999,13979206,97854442,61.7134,AAPL,14593
5,0.878571,0.042423,1.425,62.5528,2013-03-11,62.7157,"Mar 11, 13",60.7342,61.3928,16896018,118272126,61.7792,AAPL,14593
6,-1.348600,0.019949,-2.156,61.2042,2013-03-12,62.6971,"Mar 12, 13",61.0814,62.2285,16609763,116268341,61.7188,AAPL,14593
7,-0.011429,0.019759,-0.019,61.1928,2013-03-13,62.0714,"Mar 13, 13",60.7657,61.2071,14481293,101369051,61.4500,AAPL,14593
8,0.592857,0.029640,0.969,61.7857,2013-03-14,62.0914,"Mar 14, 13",61.4928,61.8328,10833558,75834906,61.7869,AAPL,14593
9,1.594300,0.056207,2.580,63.3799,2013-03-15,63.4614,"Mar 15, 13",62.4642,62.5614,22958658,160710606,62.9961,AAPL,14593


In [296]:
if 0:
    start_date = '2013-03-04'
    end_date = '2017-06-30'
    stockprice_validTime = stockprice[(stockprice['date'] >= start_date) & (stockprice['date'] <= end_date)]
    df_validTime = df[(df['date'] >= start_date) & (df['date'] <= end_date)]



In [297]:
if 0:
    a = len(np.unique(stockprice_validTime.gsid.values))
    b = len(np.unique(df_validTime.gsid.values))
    min_v = np.array([a,b]).min()
    for i in np.arange(min_v):
        if min_v == a:
            gsid_valid = np.unique(stockprice_validTime.gsid.values)
        else:
            gsid_valid = np.unique(df_validTime.gsid.values)
        # gsid_i = gsid_valid[i]
        gsid_i = 14593 # apple
        df_temp = df_validTime[df_validTime['gsid'] == str(gsid_i)]
        stockprice_temp = stockprice_validTime[stockprice_validTime['gsid'] == str(gsid_i)]
        df_temp_date = df_temp['date']
        # stockprice_temp = 
        common_date = np.intersect1d(df_temp['date'], stockprice_temp['date'])

In [303]:
df_merge = pd.merge(df, stockprice, how='inner', on=['date', 'gsid'])

In [305]:
df_merge

,date,financialReturnsScore,growthScore,gsid,integratedScore,multipleScore,updateTime,change,changeOverTime,changePercent,close,high,label,low,open,unadjustedVolume,volume,vwap,ticker
0,2013-03-04,0.208,0.696,75154,0.498,0.408,2017-05-11T19:47:36Z,0.29,0.000000,0.809,36.14,36.2000,"Mar 4, 13",35.7200,35.860,3523502,3523502,36.0210,CCL
1,2013-03-05,0.206,0.694,75154,0.494,0.418,2017-05-11T19:47:36Z,0.06,0.001660,0.166,36.20,36.5600,"Mar 5, 13",36.1000,36.300,4224201,4224201,36.3027,CCL
2,2013-03-06,0.294,0.636,75154,0.568,0.224,2017-05-11T19:47:36Z,-0.06,0.000000,-0.166,36.14,36.4000,"Mar 6, 13",36.0800,36.250,2860281,2860281,36.2478,CCL
3,2013-03-07,0.290,0.636,75154,0.566,0.226,2017-05-11T19:47:36Z,0.10,0.002767,0.277,36.24,36.4700,"Mar 7, 13",36.0601,36.070,2769930,2769930,36.3309,CCL
4,2013-03-08,0.290,0.634,75154,0.566,0.224,2017-05-11T19:47:36Z,-0.57,-0.013005,-1.573,35.67,36.3900,"Mar 8, 13",35.5200,36.200,5663099,5663099,35.8275,CCL
5,2013-03-11,0.290,0.634,75154,0.572,0.210,2017-05-11T19:47:36Z,-0.03,-0.013835,-0.084,35.64,35.9300,"Mar 11, 13",35.3700,35.480,4648216,4648216,35.6459,CCL
6,2013-03-12,0.290,0.634,75154,0.572,0.206,2017-05-11T19:47:36Z,-0.30,-0.022136,-0.842,35.34,35.6400,"Mar 12, 13",35.2600,35.590,3239703,3239703,35.4030,CCL
7,2013-03-13,0.288,0.634,75154,0.574,0.202,2017-05-11T19:47:36Z,0.37,-0.011898,1.047,35.71,35.7700,"Mar 13, 13",35.0600,35.210,3691448,3691448,35.5447,CCL
8,2013-03-14,0.290,0.636,75154,0.570,0.214,2017-05-11T19:47:36Z,0.02,-0.011345,0.056,35.73,36.1550,"Mar 14, 13",35.0100,35.250,12683939,12683939,35.4072,CCL
9,2013-03-15,0.288,0.636,75154,0.570,0.212,2017-05-11T19:47:36Z,-0.78,-0.032928,-2.183,34.95,35.2600,"Mar 15, 13",33.5400,34.040,18675536,18675536,34.6460,CCL


In [306]:
df_merge

,date,financialReturnsScore,growthScore,gsid,integratedScore,multipleScore,updateTime,change,changeOverTime,changePercent,close,high,label,low,open,unadjustedVolume,volume,vwap,ticker
0,2013-03-04,0.208,0.696,75154,0.498,0.408,2017-05-11T19:47:36Z,0.29,0.000000,0.809,36.14,36.2000,"Mar 4, 13",35.7200,35.860,3523502,3523502,36.0210,CCL
1,2013-03-05,0.206,0.694,75154,0.494,0.418,2017-05-11T19:47:36Z,0.06,0.001660,0.166,36.20,36.5600,"Mar 5, 13",36.1000,36.300,4224201,4224201,36.3027,CCL
2,2013-03-06,0.294,0.636,75154,0.568,0.224,2017-05-11T19:47:36Z,-0.06,0.000000,-0.166,36.14,36.4000,"Mar 6, 13",36.0800,36.250,2860281,2860281,36.2478,CCL
3,2013-03-07,0.290,0.636,75154,0.566,0.226,2017-05-11T19:47:36Z,0.10,0.002767,0.277,36.24,36.4700,"Mar 7, 13",36.0601,36.070,2769930,2769930,36.3309,CCL
4,2013-03-08,0.290,0.634,75154,0.566,0.224,2017-05-11T19:47:36Z,-0.57,-0.013005,-1.573,35.67,36.3900,"Mar 8, 13",35.5200,36.200,5663099,5663099,35.8275,CCL
5,2013-03-11,0.290,0.634,75154,0.572,0.210,2017-05-11T19:47:36Z,-0.03,-0.013835,-0.084,35.64,35.9300,"Mar 11, 13",35.3700,35.480,4648216,4648216,35.6459,CCL
6,2013-03-12,0.290,0.634,75154,0.572,0.206,2017-05-11T19:47:36Z,-0.30,-0.022136,-0.842,35.34,35.6400,"Mar 12, 13",35.2600,35.590,3239703,3239703,35.4030,CCL
7,2013-03-13,0.288,0.634,75154,0.574,0.202,2017-05-11T19:47:36Z,0.37,-0.011898,1.047,35.71,35.7700,"Mar 13, 13",35.0600,35.210,3691448,3691448,35.5447,CCL
8,2013-03-14,0.290,0.636,75154,0.570,0.214,2017-05-11T19:47:36Z,0.02,-0.011345,0.056,35.73,36.1550,"Mar 14, 13",35.0100,35.250,12683939,12683939,35.4072,CCL
9,2013-03-15,0.288,0.636,75154,0.570,0.212,2017-05-11T19:47:36Z,-0.78,-0.032928,-2.183,34.95,35.2600,"Mar 15, 13",33.5400,34.040,18675536,18675536,34.6460,CCL


In [319]:
np.nan()

TypeError: 'float' object is not callable